In [ ]:
%%html
<style>div.input {display:none;} div.output_stderr{display:none}</style>

In [ ]:
"""    To Start This Notebook Click On:    Cell > Run All    (in the jupyter menu above)    """
import warnings
warnings.filterwarnings('ignore')

import os
import io
import sys

import pandas as pd
from pandas.io.common import EmptyDataError
import numpy as np

from IPython.display import display, HTML
import ipywidgets as widgets
import traitlets

from knpackage import toolbox as kn

sys.path.insert(1, './src')
import spreadsheets_transformation_toolbox as stt
from layout_notebooks import *

DEFAULT_INPUT_FILES = {'transpose': 'transpose_spreadsheet.tsv', 
                       'common_samples_1': 'intersect_spreadsheet_A.tsv', 
                       'common_samples_2': 'intersect_spreadsheet_B.tsv', 
                       'merge_1': 'merge_spreadsheet_A.txt', 
                       'merge_2': 'merge_spreadsheet_B.txt',
                       'select_rows_spreadsheet': 'select_rows_spreadsheet.tsv',
                       'select_rows_list': 'select_rows_list.txt',
                       'select_averages_spreadsheet': 'average_spreadsheet.tsv',
                       'select_averages_dict': 'average_labels.tsv',
                       'select_categorical_spreadsheet': 'select_phenotype_spreadsheet.df',
                       'select_categorical_phenotype': 'select_phenotype_phenotype.txt',
                       'numerical_spreadsheet': 'other_transforms_spreadsheet.tsv',
                       'stats_spreadsheet': 'descriptive_statistic_spreadsheet.tsv',
                      }

results_dir = USER_RESULTS_DIRECTORY
input_data_dir = USER_DATA_DIRECTORY

In [ ]:
show_cell_title('Transpose')

def transpose_selected_file(button):
    """ callback for the transpose_execute_button
    
    Args: 
        button:         an ipywidgets.Button object with an ipywidgets.Dropdown (.file_selector) object containing 
                        the selected file name as its .value field.
    """
    if button.description == 'Clear':
        visualize_selected_file(button)
        return
    
    input_data_directory        = button.input_data_directory
    input_file_name             = button.file_selector.value
    spreadsheet_T_df            = stt.transpose_df(
                                    kn.get_spreadsheet_df(
                                        os.path.join(input_data_directory, input_file_name)
                                    ))

    results_directory           = button.results_directory
    transform_name              = "transpose"
    result_file_name            = stt.get_outfile_name(
                                    results_directory, 
                                    input_file_name, 
                                    transform_name, 
                                    timestamp=False,
                                    )
    spreadsheet_T_df.to_csv(result_file_name, sep='\t',float_format='%g')
    
    button.fname_list           = [result_file_name]
    visualize_selected_file(button)
    
# get the control widgets
get_transpose_file_button       = get_select_view_file_button_set(input_data_dir)
if os.path.isfile(os.path.join(input_data_dir, DEFAULT_INPUT_FILES['transpose'])):
    get_transpose_file_button.file_selector.value \
                                = DEFAULT_INPUT_FILES['transpose']

transpose_execute_button        = get_single_file_execute_button(
                                    input_data_dir, 
                                    results_dir, 
                                    file_selector=get_transpose_file_button.file_selector, 
                                    button_name='Transpose',
                                    )
transpose_execute_button.on_click(transpose_selected_file)

# display control widgets
show_select_view_button(get_transpose_file_button)
show_execute_button(transpose_execute_button)

In [ ]:
show_cell_title('Intersect')

def get_common_samples(button):
    """ callback for common_samples_execute_button
    
    Args: 
        button:         an ipywidgets.Button object with 2 ipywidgets.Dropdown (.file_selector) objects containing 
                        the selected file names in the .value fields.

    """
    if button.description == 'Clear':
        visualize_selected_file(button)
        return

    input_data_directory                = button.input_data_directory    
    spreadsheet_1_file_name             = os.path.join(input_data_directory, button.file_1_selector.value)
    spreadsheet_2_file_name             = os.path.join(input_data_directory, button.file_2_selector.value)
    transform_name                      = "common_samples"
    
    spreadsheet_1_df                    = kn.get_spreadsheet_df(spreadsheet_1_file_name)
    spreadsheet_2_df                    = kn.get_spreadsheet_df(spreadsheet_2_file_name)
    
    spreadsheet_1_df, spreadsheet_2_df  = stt.common_samples_df(spreadsheet_1_df, spreadsheet_2_df)
    
    results_directory                   = button.results_directory
    result_1_file_name                  = stt.get_outfile_name(
                                            results_directory, 
                                            spreadsheet_1_file_name, 
                                            transform_name, 
                                            timestamp=False,
                                            )
    spreadsheet_1_df.to_csv(result_1_file_name, sep='\t', float_format='%g')
    
    result_2_file_name                  = stt.get_outfile_name(
                                            results_directory, 
                                            spreadsheet_2_file_name, 
                                            transform_name, 
                                            timestamp=False,
                                            )
    spreadsheet_2_df.to_csv(result_2_file_name, sep='\t', float_format='%g')
    
    button.fname_list                   = [spreadsheet_1_file_name, spreadsheet_2_file_name]
    visualize_selected_file(button)

# get spreadsheet 1 control widgets
common_samples_flistbx_1_view_button    = get_select_view_file_button_set(input_data_dir)
if os.path.isfile(os.path.join(input_data_dir, DEFAULT_INPUT_FILES['common_samples_1'])):
    common_samples_flistbx_1_view_button.file_selector.value = DEFAULT_INPUT_FILES['common_samples_1']

# get spreadsheet 2 control widgets
common_samples_flistbx_2_view_button    = get_select_view_file_button_set(input_data_dir)
if os.path.isfile(os.path.join(input_data_dir, DEFAULT_INPUT_FILES['common_samples_2'])):
    common_samples_flistbx_2_view_button.file_selector.value = DEFAULT_INPUT_FILES['common_samples_2']

# get the execute button
file_select_dict = {'file_1_selector': common_samples_flistbx_1_view_button.file_selector,
                   'file_2_selector': common_samples_flistbx_2_view_button.file_selector}
common_samples_execute_button           = get_two_files_execute_button(
                                            input_data_dir, 
                                            results_dir, 
                                            file_select_dict=file_select_dict,
                                            button_name='Intersect')

common_samples_execute_button.on_click(get_common_samples)

# show the controls
show_select_view_button(common_samples_flistbx_1_view_button)
show_select_view_button(common_samples_flistbx_2_view_button)
show_execute_button(common_samples_execute_button)

In [ ]:
show_cell_title('Merge')

def merge_spreadsheets(button):
    """ Union: local function to read 2 spreadsheet files, find and write the common samples to 1 new spreadsheets
        using the Spreadsheets_Transformation - spreadsheets_transformation_toolbox
        
    Args: 
        button:         an ipywidgets.Button object with 2 ipywidgets.Dropdown (.file_selector) objects containing 
                        the selected file names in the .value fields.
    """
    if button.description == 'Clear':
        visualize_selected_file(button)
        return

    result_files_list = user_data_list(results_dir, FEXT='.tsv')
    
    run_parameters = {'results_directory': results_dir }
    run_parameters['spreadsheet_1_file_name'] = os.path.join(input_data_dir, button.file_1_selector.value)
    run_parameters['spreadsheet_2_file_name'] = os.path.join(input_data_dir, button.file_2_selector.value)
    stt.run_merge_df(run_parameters)
    
    file_name = set(user_data_list(results_dir, FEXT='.tsv')) - set(result_files_list)
    if len(file_name) > 0:
        fname_list = []
        for f in file_name:
            fname_list.append(os.path.join(results_dir, f))
        button.fname_list = fname_list
        visualize_selected_file(button)

merge_flistbx_1 = get_dropdown_files_listbox()
merge_flistbx_1.default = DEFAULT_INPUT_FILES['merge_1']

if os.path.isfile(os.path.join(input_data_dir, merge_flistbx_1.default)):
    merge_flistbx_1.value = merge_flistbx_1.default
    merge_flistbx_1.current = merge_flistbx_1.value
else:
    merge_flistbx_1.current = merge_flistbx_1.value
merge_flistbx_1.observe(flistbx_update, names='options')

merge_flistbx_1_dlink = traitlets.dlink((files_dropdown_main, 'options'), (merge_flistbx_1, 'options'))

merge_flistbx_1_view_button = widgets.Button(
    description='View',
    disabled=False,
    button_style='',
    tooltip='visualize selected file',
    )
merge_flistbx_1_view_button.file_selector=merge_flistbx_1
merge_flistbx_1_view_button.on_click(visualize_selected_file)
merge_flistbx_1_view_button.view_box = get_view_box()

show_select_view(merge_flistbx_1, merge_flistbx_1_view_button)
display(merge_flistbx_1_view_button.view_box)

merge_flistbx_2 = get_dropdown_files_listbox()
merge_flistbx_2.default = DEFAULT_INPUT_FILES['merge_2']

if os.path.isfile(os.path.join(input_data_dir, merge_flistbx_2.default)):
    merge_flistbx_2.value = merge_flistbx_2.default
    merge_flistbx_2.current = merge_flistbx_2.value
else:
    merge_flistbx_2.current = merge_flistbx_2.value
merge_flistbx_1.observe(flistbx_update, names='options')

merge_flistbx_2_dlink = traitlets.dlink((files_dropdown_main, 'options'), (merge_flistbx_2, 'options'))

merge_flistbx_2_view_button = widgets.Button(
    description='View',
    disabled=False,
    button_style='',
    tooltip='visualize selected file',
    )
merge_flistbx_2_view_button.file_selector=merge_flistbx_2
merge_flistbx_2_view_button.on_click(visualize_selected_file)
merge_flistbx_2_view_button.view_box = get_view_box()

show_select_view(merge_flistbx_2, merge_flistbx_2_view_button)
display(merge_flistbx_2_view_button.view_box)

In [ ]:
merge_execute_button = widgets.Button(
    description='Merge',
    disabled=False,
    button_style='',
    tooltip='merge spreadsheets',
    )
merge_execute_button.file_1_selector=merge_flistbx_1
merge_execute_button.file_2_selector=merge_flistbx_2
merge_execute_button.view_box = get_view_box()
merge_execute_button.on_click(merge_spreadsheets)

show_widget_right(merge_execute_button)
display(merge_execute_button.view_box)

In [ ]:
show_cell_title('Select Rows')

def read_a_list_file(input_file_name):
    """ local function to read a list from a text file
    
    Args:
        input_file_name:    full path name of a file containing a list
    
    Returns:
        string_input:       the list that was contained in the file
    """
    with open(input_file_name, 'r') as fh:
        str_input = fh.read()
    return list(str_input.split())

def select_genes(button):
    """ local function to read files and get common samples and write 
        using the Spreadsheets_Transformation - spreadsheets_transformation_toolbox
    
    Args: 
        button:         an ipywidgets.Button object with 2 ipywidgets.Dropdown (.file_selector) objects containing 
                        the selected file names in the .value fields.
    """
    result_files_list = user_data_list(results_dir, FEXT='.tsv')
    
    run_parameters = {'results_directory': results_dir }
    run_parameters['spreadsheet_file_name'] = os.path.join(input_data_dir, button.file_1_selector.value)
    run_parameters['gene_list_file_name'] = os.path.join(input_data_dir, button.file_2_selector.value)
    stt.run_select_genes(run_parameters)
    
    file_name = set(user_data_list(results_dir, FEXT='.tsv')) - set(result_files_list)
    if len(file_name) > 0:
        fname_list = []
        for f in file_name:
            fname_list.append(os.path.join(results_dir, f))
        button.fname_list = fname_list
        visualize_selected_file(button)

select_spreadsheet_flistbx = get_dropdown_files_listbox()
select_spreadsheet_flistbx.default = DEFAULT_INPUT_FILES['select_rows_spreadsheet']

if os.path.isfile(os.path.join(input_data_dir, select_spreadsheet_flistbx.default)):
    select_spreadsheet_flistbx.value = select_spreadsheet_flistbx.default
    select_spreadsheet_flistbx.current = select_spreadsheet_flistbx.value
else:
    select_spreadsheet_flistbx.current = select_spreadsheet_flistbx.value
select_spreadsheet_flistbx.observe(flistbx_update, names='options')

select_spreadsheet_flistbx_dlink = traitlets.dlink((files_dropdown_main, 'options'), 
                                                   (select_spreadsheet_flistbx, 'options'))

select_spreadsheet_flistbx_1_view_button = widgets.Button(
    description='View',
    disabled=False,
    button_style='',
    tooltip='visualize selected file',
    )
select_spreadsheet_flistbx_1_view_button.file_selector=select_spreadsheet_flistbx
select_spreadsheet_flistbx_1_view_button.on_click(visualize_selected_file)
select_spreadsheet_flistbx_1_view_button.view_box = get_view_box()

show_select_view(select_spreadsheet_flistbx, select_spreadsheet_flistbx_1_view_button)
display(select_spreadsheet_flistbx_1_view_button.view_box)

#                           "list of genes" file selector dropdown listbox and visualize button:
select_listfile_flistbx = get_dropdown_files_listbox()
select_listfile_flistbx.default = DEFAULT_INPUT_FILES['select_rows_list']

if os.path.isfile(os.path.join(input_data_dir, select_listfile_flistbx.default)):
    select_listfile_flistbx.value = select_listfile_flistbx.default
    select_listfile_flistbx.current = select_listfile_flistbx.value
else:
    select_listfile_flistbx.current = select_listfile_flistbx.value
select_listfile_flistbx.observe(flistbx_update, names='options')

select_listfile_flistbx_dlink = traitlets.dlink((files_dropdown_main, 'options'), 
                                                  (select_listfile_flistbx, 'options'))

select_listfile_flistbx_view_button = widgets.Button(
    description='View',
    disabled=False,
    button_style='',
    tooltip='visualize selected file',
    )
select_listfile_flistbx_view_button.file_selector=select_listfile_flistbx
select_listfile_flistbx_view_button.on_click(visualize_selected_file)
select_listfile_flistbx_view_button.view_box = get_view_box()

show_select_view(select_listfile_flistbx, select_listfile_flistbx_view_button)
display(select_listfile_flistbx_view_button.view_box)

In [ ]:
select_genes_execute_button = widgets.Button(
    description='Select',
    disabled=False,
    button_style='',
    tooltip='select genes from list',
    )
select_genes_execute_button.file_1_selector=select_spreadsheet_flistbx
select_genes_execute_button.file_2_selector=select_listfile_flistbx
select_genes_execute_button.view_box = get_view_box()
select_genes_execute_button.on_click(select_genes)

show_widget_right(select_genes_execute_button)
display(select_genes_execute_button.view_box)

In [ ]:
show_cell_title('Average on Labels')

def get_cluster_averages(button):
    """ local function to get the averages for the dictionay selected samples in a spreadsheet
        using the Spreadsheets_Transformation - spreadsheets_transformation_toolbox
    
    Args: 
        button:         an ipywidgets.Button object with 2 ipywidgets.Dropdown (.file_selector) objects containing 
                        the selected file names in the .value fields.
    """
    if button.description == 'Clear':
        visualize_selected_file(button)
        return

    result_files_list = user_data_list(results_dir, FEXT='.tsv')
    
    run_parameters = {'results_directory': results_dir }
    run_parameters['spreadsheet_file_name'] = os.path.join(input_data_dir, button.file_1_selector.value)
    run_parameters['sample_labels_file_name'] = os.path.join(input_data_dir, button.file_2_selector.value)
    run_parameters['centroid_statistic'] = button.centroid_selector.value
    
    stt.run_cluster_statistics_df(run_parameters)
    
    file_name = set(user_data_list(results_dir, FEXT='.tsv')) - set(result_files_list)
    if len(file_name) > 0:
        fname_list = []
        for f in file_name:
            fname_list.append(os.path.join(results_dir, f))
        button.fname_list = fname_list
        visualize_selected_file(button)

cluster_averages_spreadsheet_flistbx = get_dropdown_files_listbox()
cluster_averages_spreadsheet_flistbx.default = DEFAULT_INPUT_FILES['select_averages_spreadsheet']

if os.path.isfile(os.path.join(input_data_dir, cluster_averages_spreadsheet_flistbx.default)):
    cluster_averages_spreadsheet_flistbx.value = cluster_averages_spreadsheet_flistbx.default
    cluster_averages_spreadsheet_flistbx.current = cluster_averages_spreadsheet_flistbx.value
else:
    cluster_averages_spreadsheet_flistbx.current = cluster_averages_spreadsheet_flistbx.value
cluster_averages_spreadsheet_flistbx.observe(flistbx_update, names='options')

cluster_averages_spreadsheet_flistbx_dlink = traitlets.dlink((files_dropdown_main, 'options'), 
                                                   (cluster_averages_spreadsheet_flistbx, 'options'))

cluster_averages_spreadsheet_flistbx_view_button = widgets.Button(
    description='View',
    disabled=False,
    button_style='',
    tooltip='visualize selected file',
    )
cluster_averages_spreadsheet_flistbx_view_button.file_selector=cluster_averages_spreadsheet_flistbx
cluster_averages_spreadsheet_flistbx_view_button.on_click(visualize_selected_file)
cluster_averages_spreadsheet_flistbx_view_button.view_box = get_view_box()

show_select_view(cluster_averages_spreadsheet_flistbx, cluster_averages_spreadsheet_flistbx_view_button)
display(cluster_averages_spreadsheet_flistbx_view_button.view_box)

select_clusters_flistbx = get_dropdown_files_listbox()
select_clusters_flistbx.default = DEFAULT_INPUT_FILES['select_averages_dict']

if os.path.isfile(os.path.join(input_data_dir, select_clusters_flistbx.default)):
    select_clusters_flistbx.value = select_clusters_flistbx.default
    select_clusters_flistbx.current = select_clusters_flistbx.value
else:
    select_clusters_flistbx.current = select_clusters_flistbx.value
select_clusters_flistbx.observe(flistbx_update, names='options')

select_clusters_flistbx_dlink = traitlets.dlink((files_dropdown_main, 'options'), 
                                                  (select_clusters_flistbx, 'options'))

select_clusters_flistbx_view_button = widgets.Button(
    description='View',
    disabled=False,
    button_style='',
    tooltip='visualize selected file',
    )
select_clusters_flistbx_view_button.file_selector=select_clusters_flistbx
select_clusters_flistbx_view_button.on_click(visualize_selected_file)
select_clusters_flistbx_view_button.view_box = get_view_box()

show_select_view(select_clusters_flistbx, select_clusters_flistbx_view_button)
display(select_clusters_flistbx_view_button.view_box)

In [ ]:
AVAILABLE_STATISTICAL_CENTROIDS = ['std', 'median', 'mean']
centroid_selector_dropdown = widgets.Dropdown(options=AVAILABLE_STATISTICAL_CENTROIDS,
                                              value=AVAILABLE_STATISTICAL_CENTROIDS[0],
                                              description='Statistic:')

cluster_averages_execute_button = widgets.Button(description='Calculate',
                                                 disabled=False,
                                                 button_style='',
                                                 tooltip='get cluster averages')

cluster_averages_execute_button.file_1_selector=cluster_averages_spreadsheet_flistbx
cluster_averages_execute_button.file_2_selector=select_clusters_flistbx
cluster_averages_execute_button.centroid_selector=centroid_selector_dropdown
cluster_averages_execute_button.view_box = get_view_box()
cluster_averages_execute_button.on_click(get_cluster_averages)

show_widget_left(centroid_selector_dropdown)
show_widget_right(cluster_averages_execute_button)
display(cluster_averages_execute_button.view_box)

In [ ]:
show_cell_title('Select Phenotype')

def select_categorical(button):
    """ local function to read files and select categorical and write 
        using the Spreadsheets_Transformation - spreadsheets_transformation_toolbox
    
    Args: 
        button:         an ipywidgets.Button object with 2 ipywidgets.Dropdown (.file_selector) objects containing 
                        the selected file names in the .value fields.
    """
    if button.description == 'Clear':
        visualize_selected_file(button)
        return

    result_files_list = user_data_list(results_dir, FEXT='.tsv')
    
    run_parameters = {'results_directory': results_dir }
    run_parameters['spreadsheet_file_name'] = os.path.join(input_data_dir, button.spreadsheet_file_selector.value)
    run_parameters['phenotype_file_name'] = os.path.join(input_data_dir, button.phenotype_file_selector.value)
    run_parameters['phenotype_id'] = button.phenotype_id_selector.value
    run_parameters['select_category'] = button.category_selectory.value
    
    stt.run_select_subtype_df(run_parameters)
    
    file_name = set(user_data_list(results_dir, FEXT='.tsv')) - set(result_files_list)
    if len(file_name) > 0:
        fname_list = []
        for f in file_name:
            fname_list.append(os.path.join(results_dir, f))
        button.fname_list = fname_list
        visualize_selected_file(button)


""" create spreadsheet select listbox and veiw controls with default file name if available """
select_categorical_spreadsheet_flistbx = get_dropdown_files_listbox()
select_categorical_spreadsheet_flistbx.default = DEFAULT_INPUT_FILES['select_categorical_spreadsheet']

if os.path.isfile(os.path.join(input_data_dir, select_categorical_spreadsheet_flistbx.default)):
    select_categorical_spreadsheet_flistbx.value = select_categorical_spreadsheet_flistbx.default
    select_categorical_spreadsheet_flistbx.current = select_categorical_spreadsheet_flistbx.value
    DEFAULT_IS_SET = True
else:
    select_categorical_spreadsheet_flistbx.current = select_categorical_spreadsheet_flistbx.value
    DEFAULT_IS_SET = False

select_categorical_spreadsheet_flistbx.observe(flistbx_update, names='options')
select_categorical_spreadsheet_flistbx_dlink = traitlets.dlink((files_dropdown_main, 'options'), 
                                                   (select_categorical_spreadsheet_flistbx, 'options'))

select_categorical_spreadsheet_flistbx_view_button = widgets.Button(
    description='View',
    disabled=False,
    button_style='',
    tooltip='file to visualize',
    )
select_categorical_spreadsheet_flistbx_view_button.file_selector=select_categorical_spreadsheet_flistbx
select_categorical_spreadsheet_flistbx_view_button.on_click(visualize_selected_file)
select_categorical_spreadsheet_flistbx_view_button.view_box = get_view_box()

""" display: select and view spreadsheet file """
show_select_view(select_categorical_spreadsheet_flistbx, select_categorical_spreadsheet_flistbx_view_button)
display(select_categorical_spreadsheet_flistbx_view_button.view_box)


""" create phenotype select listbox and veiw controls with default file name if available """
select_phenotype_flistbx = get_dropdown_files_listbox()
select_phenotype_flistbx.default = DEFAULT_INPUT_FILES['select_categorical_phenotype']

if os.path.isfile(os.path.join(input_data_dir, select_phenotype_flistbx.default)):
    select_phenotype_flistbx.value = select_phenotype_flistbx.default
    select_phenotype_flistbx.current = select_phenotype_flistbx.value
    DEFAULT_IS_SET = True and DEFAULT_IS_SET
else:
    DEFAULT_IS_SET = False
    select_phenotype_flistbx.current = select_phenotype_flistbx.value

select_phenotype_flistbx.observe(flistbx_update, names='options')
select_phenotype_flistbx_dlink = traitlets.dlink((files_dropdown_main, 'options'), 
                                                  (select_phenotype_flistbx, 'options'))

select_phenotype_flistbx_view_button = widgets.Button(description='View',
                                                      disabled=False,
                                                      button_style='',
                                                      tooltip='file to visualize')
select_phenotype_flistbx_view_button.view_full_file = False
select_phenotype_flistbx_view_button.file_selector = select_phenotype_flistbx
select_phenotype_flistbx_view_button.on_click(visualize_selected_file)
select_phenotype_flistbx_view_button.view_box = get_view_box()

""" display: select and view phenotype file """
show_select_view(select_phenotype_flistbx, select_phenotype_flistbx_view_button)
display(select_phenotype_flistbx_view_button.view_box)



""" create dynamically coupled dropdown listboxes Such That: 
    - the first shows the list of phenotype ids COUPLED TO the phenotype file selection
    - the second shows the unique values COUPLED TO the selected phenotype id
"""
def reset_phenotype_id_list(change):
    """ update the phenotype id options and select the first one in phenotype_id_listbox, and...
        update the category_select_listbox options and select the first
    """
    new_file = os.path.join(input_data_dir, change.new)
    new_df = pd.read_csv(new_file, sep='\t', index_col=0, header=0)
    
    new_phenotpye_list = new_df.columns.tolist()
    phenotype_id_listbox.options = new_phenotpye_list
    
    phenotype_id_listbox.value = new_phenotpye_list[0]
    
    a_list = new_df[phenotype_id_listbox.value]
    a_list = a_list.dropna()
    category_select_listbox.options = sorted(set(a_list))
    category_select_listbox.value = category_select_listbox.options[0]

""" - the first shows the list of phenotype ids COUPLED TO the phenotype file selection """
phenotype_id_listbox = widgets.Dropdown(options=[''],
                                        value='',
                                        description='column:')
select_phenotype_flistbx.target_dir=input_data_dir
select_phenotype_flistbx.observe(reset_phenotype_id_list, names='value')

def reset_category_list(change):
    """ update the category_select_listbox options and select the first option
    """
    new_file = os.path.join(input_data_dir, select_phenotype_flistbx.value)
    new_df = pd.read_csv(new_file, sep='\t', index_col=0, header=0)
    
    a_list = new_df[phenotype_id_listbox.value]
    a_list = a_list.dropna()
    
    category_select_listbox.options = sorted(set(a_list))
    category_select_listbox.value = category_select_listbox.options[0]

""" - the second shows the unique values COUPLED TO the selected phenotype id """
category_select_listbox = widgets.Dropdown(options=[''],
                                           value='',
                                           description='phenotype:')
phenotype_id_listbox.observe(reset_category_list, names='value')

"""  """
if DEFAULT_IS_SET == True:
    select_phenotype_flistbx.value = select_phenotype_flistbx.options[0]
    select_phenotype_flistbx.value = select_phenotype_flistbx.current
    phenotype_id_listbox.value = phenotype_id_listbox.options[1]
    phenotype_id_listbox.value = phenotype_id_listbox.options[0]

""" display:  The phenotype category_id and category controls """
show_widget_left(widgets.VBox([phenotype_id_listbox, category_select_listbox]))

In [ ]:
select_categorical_execute_button = widgets.Button(
    description='Select',
    disabled=False,
    button_style='',
    tooltip='select category in both',
)
select_categorical_execute_button.spreadsheet_file_selector=select_categorical_spreadsheet_flistbx
select_categorical_execute_button.phenotype_file_selector=select_phenotype_flistbx
select_categorical_execute_button.phenotype_id_selector=phenotype_id_listbox
select_categorical_execute_button.category_selectory=category_select_listbox
select_categorical_execute_button.view_box = get_view_box()
select_categorical_execute_button.on_click(select_categorical)

show_widget_right(select_categorical_execute_button)
display(select_categorical_execute_button.view_box)

In [ ]:
show_cell_title('Other Transformations')

def select_numerical_transform(button):
    """ callback for get_numerical_function_execute_button
    
    Args: 
        button:         an ipywidgets.Button object with 2 ipywidgets.Dropdown (.file_selector) objects containing 
                        the selected file names in the .value fields.
    """
    if button.description == 'Clear':
        visualize_selected_file(button)
        return
    
    input_data_directory            = button.input_data_directory
    input_file_name                 = button.file_selector.value
    spreadsheet_df                  = kn.get_spreadsheet_df(os.path.join(input_data_directory, input_file_name))

    numeric_function                = button.numerical_function.value
    results_directory               = button.results_directory
    transform_name                  = numeric_function
    
    if transform_name == 'abs':
        result_df                   = stt.abs_df(spreadsheet_df)
        
    elif transform_name == 'z_transform':
        z_transform_axis            = button.numerical_function.z_transform_axis.value
        z_transform_ddof            = button.numerical_function.z_transform_ddof.value
        result_df                   = stt.z_transform_df(
                                        spreadsheet_df, 
                                        axis=z_transform_axis, 
                                        ddof=z_transform_ddof,
                                        )
        
    elif transform_name == 'log_transform':
        log_transform_log_base      = button.numerical_function.log_transform_log_base.value
        if log_transform_log_base == "e":
            log_transform_log_base  = np.exp(1)
        log_transform_log_offset    = button.numerical_function.log_transform_log_offset.value
        result_df                   = stt.log_transform_df(spreadsheet_df,
                                        log_base=log_transform_log_base,
                                        log_offset=log_transform_log_offset,
                                        )
        
    elif transform_name == 'threshold':
        threshold_cut_off               = button.numerical_function.threshold_cut_off.value
        threshold_substitution_value    = button.numerical_function.threshold_substitution_value.value
        threshold_scope                 = button.numerical_function.threshold_scope.value
        result_df                       = stt.threshold_df(spreadsheet_df,
                                            cut_off=threshold_cut_off,
                                            sub_val=threshold_substitution_value,
                                            scope=threshold_scope,
                                            )

    result_file_name = stt.get_outfile_name(results_directory, input_file_name, transform_name, timestamp=False)
    result_df.to_csv(result_file_name, sep='\t',float_format='%g')
    button.fname_list = [result_file_name]
    visualize_selected_file(button)

# get the control widgets
get_other_transform_file_button         = get_select_view_file_button_set(input_data_dir)
if os.path.isfile(os.path.join(input_data_dir, DEFAULT_INPUT_FILES['numerical_spreadsheet'])):
    get_other_transform_file_button.file_selector.value = DEFAULT_INPUT_FILES['numerical_spreadsheet']
    
""" display a numeric function options listbox and if selected is:
    abs             - grey out all else in all cases
    z_transform     - show radio button row or columns, and a int text box for ddof
    log_transform   - show log_transform float base text box, and a float offset box
    threshold       - show a float threshold cut-off, substitution box, and threshold bound [SUB_BELOW, SUB_ABOVE]
"""
def reset_aux_controls(change):
    """ callback - sub controls setup for numerical_transform method options """
    if change.old == 'z_transform':
        change['owner'].z_transform_axis.disabled=True
        change['owner'].z_transform_ddof.disabled=True
        
    elif change.old == 'log_transform':
        change['owner'].log_transform_log_base.disabled=True
        change['owner'].log_transform_log_offset.disabled=True
        
    elif change.old == 'threshold':
        change['owner'].threshold_cut_off.disabled=True
        change['owner'].threshold_substitution_value.disabled=True
        change['owner'].threshold_scope.disabled=True
    
    if change.new == 'abs':
        pass
    
    elif change.new == 'z_transform':
        change['owner'].z_transform_axis.disabled=False
        change['owner'].z_transform_ddof.disabled=False
        
    elif change.new == 'log_transform':
        change['owner'].log_transform_log_base.disabled=False
        change['owner'].log_transform_log_offset.disabled=False

    elif change.new == 'threshold':
        change['owner'].threshold_cut_off.disabled=False
        change['owner'].threshold_substitution_value.disabled=False
        change['owner'].threshold_scope.disabled=False

numerical_function_options                          = ['abs', 'z_transform', 'log_transform', 'threshold']
numerical_function_dropdown                         = widgets.Dropdown(
                                                        options=numerical_function_options,
                                                        value=numerical_function_options[0],
                                                        description='stats function')

threshold_scope                                          = ['SUB_BELOW', 'SUB_ABOVE']
log_trans_base                                           = np.exp(1)
default_threshold                                        = 0.5
numerical_function_dropdown.z_transform_axis             = widgets.Dropdown(
                                                            options={'rows': 0, 'columns': 1},
                                                            value=0, description='axis', 
                                                            disabled=True)
numerical_function_dropdown.z_transform_ddof             = widgets.IntText(
                                                            value=0,
                                                            description='ddof',
                                                            disabled=True)
numerical_function_dropdown.log_transform_log_base       = widgets.FloatText(
                                                            value=log_trans_base,
                                                            description='log_base', 
                                                            disabled=True)
numerical_function_dropdown.log_transform_log_offset     = widgets.FloatText(
                                                            value=0,
                                                            description='offset', 
                                                            disabled=True)
numerical_function_dropdown.threshold_cut_off            = widgets.FloatText(
                                                            value=default_threshold,
                                                            description='cut-off', 
                                                            disabled=True)
numerical_function_dropdown.threshold_substitution_value = widgets.FloatText(
                                                            value=0,
                                                            description='substitute', 
                                                            disabled=True)
numerical_function_dropdown.threshold_scope              = widgets.Dropdown(
                                                            options=threshold_scope,
                                                            value=threshold_scope[0],
                                                            description='sub', disabled=True)
numerical_function_dropdown.observe(reset_aux_controls, names='value')

get_numerical_function_execute_button               = get_single_file_execute_button(
                                                        input_data_dir, 
                                                        results_dir, 
                                                        file_selector=get_other_transform_file_button.file_selector, 
                                                        button_name='Calculate',
                                                        )
get_numerical_function_execute_button.numerical_function = numerical_function_dropdown
get_numerical_function_execute_button.on_click(select_numerical_transform)

# display control widgets
show_select_view_button(get_other_transform_file_button)
show_widget_left(numerical_function_dropdown)
show_widget_left(widgets.VBox([
                        numerical_function_dropdown.z_transform_axis,
                        numerical_function_dropdown.z_transform_ddof,
                        numerical_function_dropdown.log_transform_log_base,
                        numerical_function_dropdown.log_transform_log_offset,
                        numerical_function_dropdown.threshold_cut_off,
                        numerical_function_dropdown.threshold_substitution_value,
                        numerical_function_dropdown.threshold_scope,
                        ]))
show_execute_button(get_numerical_function_execute_button)

In [ ]:
show_cell_title('Descriptive Statistics')

def get_stats_value(button):
    """ callback for get_stats_execute_button
    
    Args: 
        button:         an ipywidgets.Button object with 2 ipywidgets.Dropdown (.file_selector) objects containing 
                        the selected file names in the .value fields.
    """
    if button.description == 'Clear':
        visualize_selected_file(button)
        return

    input_data_directory                        = button.input_data_directory
    input_file_name                             = button.file_selector.value
    spreadsheet_df                              = kn.get_spreadsheet_df(
                                                    os.path.join(input_data_directory, input_file_name)
                                                    )
    stats_function                              = button.stats_function.value
    direction_reference                         = button.direction_reference.value
    
    result_df                                   = stt.stats_df(
                                                    spreadsheet_df, 
                                                    stats_function, 
                                                    direction_reference,
                                                    )
    results_directory                           = button.results_directory
    stats_function                              = button.stats_function.value
    direction_reference                         = button.direction_reference.value
    transform_name                              = stats_function + '_' + direction_reference
    
    result_file_name                            = stt.get_outfile_name(
                                                    results_directory, 
                                                    input_file_name, 
                                                    transform_name, 
                                                    timestamp=False,
                                                    )
    result_df.to_csv(result_file_name, sep='\t',float_format='%g')
    
    button.fname_list                           = [result_file_name]
    visualize_selected_file(button)

# get the control widgets
get_stats_function_file_button                  = get_select_view_file_button_set(input_data_dir)

if os.path.isfile(os.path.join(input_data_dir, DEFAULT_INPUT_FILES['stats_spreadsheet'])):
    get_stats_function_file_button.file_selector.value \
                                                = DEFAULT_INPUT_FILES['stats_spreadsheet']

stats_function_options                          = {
                                                    'min': 'min',
                                                    'max': 'max', 
                                                    'mean': 'mean', 
                                                    'median': 'median', 
                                                    'variance': 'variation', 
                                                    'std_deviation': 'std_deviation', 
                                                    'sum': 'sum',
                                                    }

stats_function_dropdown                         = widgets.Dropdown(
                                                    options=stats_function_options,
                                                    value=stats_function_options['min'],
                                                    description='function',
                                                    )

direction_reference_options                     = ['columns', 'rows', 'all']
direction_reference_dropdown                    = widgets.Dropdown(
                                                    options=direction_reference_options,
                                                    value=direction_reference_options[0],
                                                    description='direction',
                                                    )

get_stats_execute_button                        = get_single_file_execute_button(
                                                    input_data_dir, 
                                                    results_dir, 
                                                    file_selector=get_stats_function_file_button.file_selector, 
                                                    button_name='Calculate',
                                                    )
get_stats_execute_button.stats_function         = stats_function_dropdown
get_stats_execute_button.direction_reference    = direction_reference_dropdown
get_stats_execute_button.on_click(get_stats_value)

# display control widgets
show_select_view_button(get_stats_function_file_button)
show_widget_left(widgets.VBox([stats_function_dropdown, direction_reference_dropdown]))
show_execute_button(get_stats_execute_button)